![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, Elasticsearch, and LangChain to answer questions (RAG)

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.

## Notebook content

This notebook contains the steps and code to demonstrate support of Retrieval Argumented Generation in watsonx.ai. It introduces commands for data retrieval, knowledge base building & querying, and model testing. 

Following this, an AI service is created based on the previously constructed application.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

#### About Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

In its simplest form, RAG requires 3 steps:

- Index knowledge base passages (once)
- Retrieve relevant passage(s) from knowledge base (for every user query)
- Generate a response by feeding retrieved passage into a large language model (for every user query)

## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Test data loading](#data)
- [Foundation Models on watsonx](#models)
- [Set up connectivity information to Elasticsearch](#elastic_conn)
- [Generate a retrieval-augmented response to a question](#predict)
- [Create AI service](#ai_service)
- [Testing AI service's function locally](#testing)
- [Deploy AI service](#deploy)
- [Example of Executing an AI service](#example)
- [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install wget | tail -n 1
!pip install pandas | tail -n 1
!pip install humanize | tail -n 1
!pip install -U "langchain>=0.3,<0.4" | tail -n 1
!pip install -U "ibm_watsonx_ai>=1.1.22" | tail -n 1
!pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1
!pip install -U "langchain-huggingface>=0.1,<0.2" | tail -n 1
!pip install -U "langchain-elasticsearch>=0.3,<0.4" | tail -n 1

In [1]:
import os
import random
import getpass
import pandas as pd
import humanize

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [3]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press **Create**
- Go to **Manage** tab
- Copy `Space GUID` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below


In [4]:
try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your project_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [3]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(credentials=credentials, space_id=space_id)

<a id="data"></a>
## Test data loading

Download the test dataset. This dataset is used to calculate the metrics score for selected model, defined prompts and parameters.

In [4]:
import wget

questions_test_filename = 'questions_test.csv'
questions_train_filename = 'questions_train.csv'
questions_test_url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/RAG/questions_test.csv'
questions_train_url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/RAG/questions_train.csv'


if not os.path.isfile(questions_test_filename): 
    wget.download(questions_test_url, out=questions_test_filename)


if not os.path.isfile(questions_train_filename): 
    wget.download(questions_train_url, out=questions_train_filename)

In [5]:
filename_test = './questions_test.csv'
filename_train =  './questions_train.csv'

test_data = pd.read_csv(filename_test)
train_data = pd.read_csv(filename_train)

Inspect data sample

In [6]:
train_data.head()

,qid,question,answers
0,1961,where does diffusion occur in the excretory sy...,diffusion
1,7528,when did the us join world war one,"April 6 , 1917"
2,8685,who played wilma in the movie the flintstones,Elizabeth Perkins
3,6716,when was the office of the vice president created,1787
4,2916,where does carbon fixation occur in c4 plants,in the mesophyll cells


### Build up knowledge base

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

We can generate dense vector representations using embedding models. In this notebook, we use `all-MiniLM-L6-v2` to embed both the knowledge base passages and user queries.

A vector database is optimized for dense vector indexing and retrieval. This notebook uses <a href="https://python.langchain.com/docs/integrations/vectorstores/elasticsearch#basic-example" target="_blank" rel="noopener no referrer">Elasticsearch</a>, a distributed, RESTful search and analytics engine, capable of performing both vector and lexical search. It is built on top of the Apache Lucene library, which offers good speed and performance with `all-MiniLM-L6-v2` embedding model.

### Load knowledge base documents

Load set of documents used further to build knowledge base. 

In [7]:
knowledge_base_dir = "./knowledge_base"

my_path = f"{os.getcwd()}/knowledge_base"
if not os.path.isdir(my_path):
   os.makedirs(my_path)
   
documents_filename = 'knowledge_base/psgs.tsv'
documents_url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/RAG/psgs.tsv'

if not os.path.isfile(documents_filename): 
    wget.download(documents_url, out=documents_filename)

In [8]:
documents = pd.read_csv(f"{knowledge_base_dir}/psgs.tsv", sep='\t', header=0, nrows=1000)
documents['indextext'] = documents['title'].astype(str) + "\n" + documents['text']

In [9]:
from langchain_core.documents import Document

lc_documents = [Document(page_content=text, metadata={"id": doc_id}) 
          for text, doc_id in zip(documents['indextext'], documents['id'])]

### Create an embedding function

Note that you can feed a custom embedding function to be used by Elasticsearch. The performance of Elasticsearch may differ depending on the embedding model used.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

emb_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<a id="models"></a>
## Foundation Models on watsonx

### Defining model
You need to specify `model_id` that will be used for inferencing:

In [11]:
text_model_id = api_client.foundation_models.TextModels.FLAN_UL2

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [12]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

### Initialize the `WatsonxLLM` class.

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the <a href="https://python.langchain.com/docs/how_to/custom_llm/" target="_blank" rel="noopener no referrer">LangChain documentation</a>

In [13]:
from langchain_ibm import WatsonxLLM

watsonx_llm = WatsonxLLM(
    model_id=text_model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    space_id=space_id,
    params=parameters
)

<a id="elastic_conn"></a>
## Set up connectivity information to Elasticsearch

**This notebook focuses on self-managed cluster using <a href="https://cloud.ibm.com/docs/databases-for-elasticsearch?topic=databases-for-elasticsearch-getting-started" target="_blank" rel="noopener no referrer">IBM Cloud® Databases for Elasticsearch.</a>**

The following cell retrieves the Elasticsearch url, username, password and ssl_certificate (base_64) and prompts you to provide them manually in case of failure.

You can provide a connection asset ID to read all required connection data from it. Before doing so, make sure that a connection asset was created in your space.

In [16]:
elasticsearch_connection_id = input("Provide connection asset ID in your space. Skip this, if you wish to type credentials by hand and hit enter: ") or None

if elasticsearch_connection_id is None:
    
    elasticsearch_url = input("Please enter your Elasticsearch url name and hit enter: ")
    username = input("Please enter your Elasticsearch user name and hit enter: ")
    password = getpass.getpass("Please enter your Elasticsearch password and hit enter: ")
    ssl_certificate = input("Please enter your Elasticsearch ssl certificate (base64) and hit enter: ")

    elasticsearch_data_source_type_id = api_client.connections.get_datasource_type_uid_by_name(
        "elasticsearch"
    )
    connections_details = api_client.connections.create(
        {
            api_client.connections.ConfigurationMetaNames.NAME: "elasticsearch Connection",
            api_client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection created by the sample notebook",
            api_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: elasticsearch_data_source_type_id,
            api_client.connections.ConfigurationMetaNames.PROPERTIES: {
                "url": elasticsearch_url,
                "username": username,
                "password": password,
                "use_anonymous_access": "false",
                "ssl_certificate": ssl_certificate,
            },
        }
    )

    elasticsearch_connection_id = api_client.connections.get_id(connections_details)

Creating connections...
SUCCESS


We first create a regular Elasticsearch Python client connection using watsonx's VectorStore comeponent. 

In [17]:
from datetime import datetime

index_name = f"elastic_index_{datetime.now().strftime('%Y_%m_%d_%H%M%S')}"
index_name

'elastic_index_2024_12_02_102325'

In [18]:
from ibm_watsonx_ai.foundation_models.extensions.rag.vector_stores.vector_store import VectorStore

knowledge_base = VectorStore(
    api_client,
    connection_id=elasticsearch_connection_id,
    embeddings=emb_func,
    index_name=index_name,
)

elasticsearch_client = knowledge_base.get_client().client

<a id="elasticsearchstore_index"></a>
### Embed and index documents with Elasticsearch

**Note: Could take several minutes if you don't have pre-built indices**

In [ ]:
stored_documents = knowledge_base.add_documents(content=lc_documents)

Let's take a look in Elasticsearch what the LangChain wrapper has created. First we display the newly created index ("tables" in Elasticsearch are always called "index"). Note the field `vector` of type `dense_vector` with `dot_product` similarity.

In [21]:
dict(elasticsearch_client.indices.get(index=index_name))

{'elastic_index_2024_12_02_102325': {'aliases': {},
  'mappings': {'properties': {'metadata': {'properties': {'id': {'type': 'long'}}},
    'text': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vector': {'type': 'dense_vector', 'dims': 384}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'allocation': {'max_retries': '15'},
    'number_of_shards': '1',
    'provided_name': 'elastic_index_2024_12_02_102325',
    'creation_date': '1733131414207',
    'unassigned': {'node_left': {'delayed_timeout': '60m'}},
    'number_of_replicas': '1',
    'uuid': 'LKyI2AYfRySDMIzZOhGf0g',
    'version': {'created': '8070099'}}}}}

Verify the number of documents loaded into the Elasticsearch index.

In [23]:
doc_count = knowledge_base.count()
doc_count

1000

Let's retrieve a random document as a sample. Note the embedding in the vector field, that was generated with the watsonx embedding model.

In [ ]:
elasticsearch_client.search(index=index_name).get("hits").get("hits")[random.randint(0, elasticsearch_client.search(index=index_name).get("took"))]

Display the total size and indexing time of the new index in Elasticsearch.

In [25]:
index_stats = elasticsearch_client.indices.stats(index=index_name).get('_all').get('primaries')
print("Index size:    " + humanize.naturalsize(index_stats.get('store').get('size_in_bytes')))
print("Indexing time: " + humanize.precisedelta(index_stats.get('indexing').get('index_time_in_millis')/1000, minimum_unit='minutes'))

Index size:    9.4 MB
Indexing time: 0 minutes


<a id="predict"></a>
## Generate a retrieval-augmented response to a question

`RetrievalQA` is a chain to do question answering.

### Select questions

The prompts we will use to test the RAG flow

In [26]:
questions_and_answers = {
    "names of founding fathers of the united states?":"Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton",
    "who played in the super bowl in 2013?":"Baltimore Ravens::San Francisco 49ers",
    "when did bucharest become the capital of romania?":"1862"
}

### Retrieve relevant context

Fetch paragraphs similar to the question

In [27]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=watsonx_llm, chain_type="stuff", retriever=knowledge_base.as_langchain_retriever(), return_source_documents=True)

In [28]:
results_1 = []

for question in questions_and_answers.keys():
    result = qa.invoke({"query": question})
    results_1.append(result)

Get the set of chunks for one of the questions.

In [29]:
for idx, result in enumerate(results_1):
    print("=========")
    print("Question = ", result['query'])
    print("Answer = ", result['result'])
    print("Expected Answer(s) (may not be appear with exact wording in the dataset) = ", questions_and_answers[result['query']])
    print("\n")
    print("Source documents:")
    print(*(x.page_content for x in result['source_documents']), sep='\n')
    print("\n")

Question =  names of founding fathers of the united states?
Answer =  John Adams , Benjamin Franklin , Alexander Hamilton , John Jay , Thomas Jefferson , James Madison , and George Washington
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton


Source documents:
Founding Fathers of the United States
^ Burstein , Andrew . `` Politics and Personalities : Garry Wills takes a new look at a forgotten founder , slavery and the shaping of America '' , Chicago Tribune ( November 09 , 2003 ) : `` Forgotten founders such as Pickering and Morris made as many waves as those whose faces stare out from our currency . '' ^ Jump up to : Rafael , Ray . The Complete Idiot 's Guide to the Founding Fathers : And the Birth of Our Nation ( Penguin , 2011 ) . Jump up ^ `` Founding Fathers : Virginia '' . FindLaw Constitutional Law Center . 2008 . Retrieved 2008 - 11 - 14 . 

<a id="ai_service"></a>
## Create AI service

Prepare function which will be deployed using AI service.

In [ ]:
def deployable_ai_service(context, space_id=space_id, url=credentials["url"], elasticsearch_connection_id=elasticsearch_connection_id, index_name=index_name, text_model_id=text_model_id, **kwargs):
    
    from ibm_watsonx_ai.foundation_models.extensions.rag.vector_stores.vector_store import VectorStore
    from ibm_watsonx_ai import APIClient, Credentials
    from langchain.chains.retrieval_qa.base import RetrievalQA
    from langchain_huggingface import HuggingFaceEmbeddings
    from langchain_ibm import WatsonxLLM
    from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
    from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

    api_client = APIClient(
        credentials=Credentials(url=url, token=context.generate_token()),
        space_id=space_id
    )

    wx_parameters = {
        GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 50
    }

    watsonx_llm = WatsonxLLM(
        model_id=text_model_id,
        watsonx_client=api_client,
        params=wx_parameters
    )

    emb_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    knowledge_base = VectorStore(
        api_client=api_client,
        connection_id=elasticsearch_connection_id,
        embeddings=emb_func,
        index_name=index_name,
    )

    qa = RetrievalQA.from_chain_type(llm=watsonx_llm, chain_type="stuff", retriever=knowledge_base.as_langchain_retriever(), return_source_documents=True)

    def generate(context) -> dict:
        
        api_client.set_token(context.get_token())
   
        payload = context.get_json()
        questions_and_answers = payload["questions_and_answers"]
        results = []
        for question in questions_and_answers.keys():
            result = qa.invoke({"query": question})
            results.append(result)
        
        for res in results:
            for ind in range(len(res["source_documents"])):
                res["source_documents"][ind] = res["source_documents"][ind].to_json()
        
        response_body = {
            "query": questions_and_answers,
            "result": results,
        }
        return {"body": response_body}
        
    return generate

There is also possibility to create streaming service

```python
def deployable_ai_service(context, **custom):
    ...
    
    def generate(context) -> dict:
        ...
        return {"body": ...}

    def generate_stream(context) -> Iterator:
        ...
        yield ...
        
    return generate, generate_stream

<a id="testing"></a>
## Testing AI service's function locally

You can test AI service function locally. Initialise `RuntimeContext` firstly.

In [31]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=api_client)

In [ ]:
context.request_payload_json = {"questions_and_answers": questions_and_answers}

resp = deployable_ai_service(context=context)(context)

In [33]:
for idx, result in enumerate(resp["body"]["result"]):
    print("=========")
    print("Question = ", result['query'])
    print("Answer = ", result['result'])
    print("Expected Answer(s) (may not be appear with exact wording in the dataset) = ", questions_and_answers[result['query']])
    print("\n")
    print("Source documents:")
    print(*(x.get("kwargs").get("page_content") for x in result['source_documents']), sep='\n')
    print("\n")

Question =  names of founding fathers of the united states?
Answer =  John Adams , Benjamin Franklin , Alexander Hamilton , John Jay , Thomas Jefferson , James Madison , and George Washington
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton


Source documents:
Founding Fathers of the United States
^ Burstein , Andrew . `` Politics and Personalities : Garry Wills takes a new look at a forgotten founder , slavery and the shaping of America '' , Chicago Tribune ( November 09 , 2003 ) : `` Forgotten founders such as Pickering and Morris made as many waves as those whose faces stare out from our currency . '' ^ Jump up to : Rafael , Ray . The Complete Idiot 's Guide to the Founding Fathers : And the Birth of Our Nation ( Penguin , 2011 ) . Jump up ^ `` Founding Fathers : Virginia '' . FindLaw Constitutional Law Center . 2008 . Retrieved 2008 - 11 - 14 . 

<a id="deploy"></a>
## Deploy AI service

Prepare a configuration file for defining a custom software specification.

In [34]:
config_yml =\
"""
name: python311
channels:
  - empty
dependencies:
  - pip:
    - langchain-huggingface==0.1.2
prefix: /opt/anaconda3/envs/python311
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [35]:
base_sw_spec_id = api_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_prop_pkg_extn = {
    api_client.package_extensions.ConfigurationMetaNames.NAME: "langchain watsonx.ai env",
    api_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with langchain",
    api_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = api_client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_id = api_client.package_extensions.get_id(pkg_extn_details)
pkg_extn_id

Creating package extensions
SUCCESS


'3cce971c-4805-4925-9776-ad484f974d06'

In [36]:
meta_prop_sw_spec = {
    api_client.software_specifications.ConfigurationMetaNames.NAME: "AI service watsonx.ai custom software specification",
    api_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for AI service deployment",
    api_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_id}
}

sw_spec_details = api_client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = api_client.software_specifications.get_id(sw_spec_details)
api_client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)
sw_spec_id

SUCCESS


'a5c002e4-8022-4341-91b6-aabcf5b727de'

In [37]:
meta_props = {
    api_client.repository.AIServiceMetaNames.NAME: "AI service SDK",    
    api_client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = api_client.repository.store_ai_service(deployable_ai_service, meta_props)

In [38]:
ai_service_id = api_client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

'094b6d58-8bd4-4dfd-95da-7660636cb33b'

# Deploy AI service

Create deployment of AI service.

In [ ]:
meta_props = {
    api_client.deployments.ConfigurationMetaNames.NAME: "AI service elasticsearch",
    api_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = api_client.deployments.create(ai_service_id, meta_props)



######################################################################################

Synchronous deployment creation for id: '094b6d58-8bd4-4dfd-95da-7660636cb33b' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
.......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='575e0d2d-02eb-4195-bd38-b5fd94af32ee'
-----------------------------------------------------------------------------------------------




In [40]:
deployment_id = api_client.deployments.get_id(deployment_details)

<a id="example"></a>
## Example of Executing an AI service.

In [42]:
deployments_results = api_client.deployments.run_ai_service(
    deployment_id, {"questions_and_answers": questions_and_answers}
)

In [43]:
for idx, result in enumerate(deployments_results["result"]):
    print("=========")
    print("Question = ", result['query'])
    print("Answer = ", result['result'])
    print("Expected Answer(s) (may not be appear with exact wording in the dataset) = ", questions_and_answers[result['query']])
    print("\n")
    print("Source documents:")
    print(*(x.get("kwargs").get("page_content") for x in result['source_documents']), sep='\n')
    print("\n")

Question =  names of founding fathers of the united states?
Answer =  John Adams , Benjamin Franklin , Alexander Hamilton , John Jay , Thomas Jefferson , James Madison , and George Washington
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton


Source documents:
Founding Fathers of the United States
^ Burstein , Andrew . `` Politics and Personalities : Garry Wills takes a new look at a forgotten founder , slavery and the shaping of America '' , Chicago Tribune ( November 09 , 2003 ) : `` Forgotten founders such as Pickering and Morris made as many waves as those whose faces stare out from our currency . '' ^ Jump up to : Rafael , Ray . The Complete Idiot 's Guide to the Founding Fathers : And the Birth of Our Nation ( Penguin , 2011 ) . Jump up ^ `` Founding Fathers : Virginia '' . FindLaw Constitutional Law Center . 2008 . Retrieved 2008 - 11 - 14 . 

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.